In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os


In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
        

        

In [7]:
data_dir = r'C:\Users\ACER\Desktop\Pytorch_Learning\image_classification_resnet\Flowers_Classification_dataset'
image_datasets= {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

In [9]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)

class_names = image_datasets['train'].classes
class_names

{'train': 2000, 'val': 364}


['daisy', 'dandelion']

In [11]:
model = models.resnet18(pretrained = True)
for name, param in model.named_parameters():
    if "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum =0.9)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

C:\Users\ACER\anacond3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ACER\anacond3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\ACER/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:28<00:00, 1.62MB/s]


In [13]:
num_epochs = 10
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training complete!")

train Loss: 0.7711 Acc: 0.7230
val Loss: 0.2163 Acc: 0.9121
train Loss: 0.5032 Acc: 0.8010
val Loss: 0.5305 Acc: 0.8077
train Loss: 0.5214 Acc: 0.7985
val Loss: 0.2429 Acc: 0.9121
train Loss: 0.4851 Acc: 0.8035
val Loss: 0.2117 Acc: 0.9231
train Loss: 0.5231 Acc: 0.8005
val Loss: 0.3816 Acc: 0.8516
train Loss: 0.4494 Acc: 0.8295
val Loss: 0.2137 Acc: 0.9258
train Loss: 0.4729 Acc: 0.8155
val Loss: 0.1961 Acc: 0.9313
train Loss: 0.4564 Acc: 0.8235
val Loss: 0.2044 Acc: 0.9258
train Loss: 0.5682 Acc: 0.8005
val Loss: 0.3183 Acc: 0.9038
train Loss: 0.5011 Acc: 0.8070
val Loss: 0.2093 Acc: 0.9313
Training complete!


In [14]:
torch.save(model.state_dict(), 'flower_classification_model.pth')